In [260]:
from dotenv import load_dotenv

import numpy as np
import pandas as pd
import os, boto3, io

BUCKET = os.getenv("BUCKET")
PROCESSED_DATA_FOLDER = os.getenv("PROCESSED_DATA_FOLDER")
PATCH = "patch_15_6/"
CHAMPION_X_ROLE = "champion_x_role_sample.csv"

In [261]:
def get_processed_data_file(granularity: str) -> pd.DataFrame:

    if granularity == "single_user":
        key = f"{PROCESSED_DATA_FOLDER}/champion_x_role_x_user/{PATCH}/single_user_aggregated_data.csv"
    elif granularity == "champion_x_role":
        key = f"{PROCESSED_DATA_FOLDER}/champion_x_role/{PATCH}/champion_x_role_aggregated_data.csv"
    elif granularity == "champion_x_role_x_user":
        key = f"{PROCESSED_DATA_FOLDER}/single_user_data/{PATCH}/champion_x_role_x_user_aggregated_data.csv"
    else:
        raise ValueError("Incorrect granularity input, must be 'champion_x_role', 'champion_x_role_x_user' or 'single_user'")
    
    # Pull the object
    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=BUCKET, Key=key)

    # Read it straight into pandas
    df = pd.read_csv(io.BytesIO(obj["Body"].read()))

    print(df.shape)

    return df


In [262]:
pd.set_option("display.max_columns", None)

champion_x_role_df = pd.read_csv(CHAMPION_X_ROLE)
display(champion_x_role_df)

,champion_id,champion_name,team_position,total_games_played_in_role,total_games_per_champion,avg_kills,avg_deaths,avg_deaths_by_enemy_champs,avg_assists,avg_kill_participation,avg_takedowns,total_wins,pct_of_games_with_highest_damage_dealt,avg_pct_damage_dealt_in_team,average_damage_per_minute,avg_damage_dealt_to_champions,avg_total_damage_dealt,avg_magic_damage_dealt_to_champions,avg_total_magic_damage_dealt,avg_physical_damage_dealt_to_champions,avg_total_physical_damage_dealt,avg_true_damage_dealt_to_champions,avg_total_true_damage_dealt,avg_largest_critical_strike,avg_pct_damage_taken_in_team,avg_damage_taken,avg_magic_damage_taken,avg_physical_damage_taken,avg_true_damage_taken,avg_damage_self_mitigated,avg_times_killed_champ_took_full_team_damage_and_survived,avg_times_survived_single_digit_hp,avg_times_survived_three_immobilizes_in_fight,avg_times_took_large_damage_survived,pct_of_games_with_highest_crowd_control_score,avg_time_ccing_others,avg_times_applied_cc_on_others,avg_enemy_champion_immobilizations,avg_total_healing,avg_heals_on_teammate,avg_total_units_healed,avg_dmg_shielded_on_team,avg_effective_heal_and_shield,total_games_completed_supp_quest_first,avg_supp_quest_completion_time,avg_longest_time_spent_alive,avg_time_spent_dead,avg_spell1_casts,avg_spell2_casts,avg_spell3_casts,avg_spell4_casts,avg_ability_uses,avg_times_dodged_skillshot_in_small_window,avg_skillshots_dodged,avg_skillshots_landed_early_game,avg_skillshots_hit,avg_times_immobilize_and_kill_with_ally,avg_times_got_kill_after_hidden_with_ally,avg_times_pick_kill_with_ally,avg_times_knock_enemy_into_team_and_kill,avg_kills_near_enemy_turret,avg_kills_under_own_turret,avg_multikills_after_aggressive_flash,avg_outnumbered_kills,avg_times_outnumbered_nexus_kill,avg_times_quick_cleanse,avg_quick_solo_kills,avg_solo_kills,avg_takedowns_after_gaining_lvl_advantage,avg_kills_on_other_lanes_early_as_laner,avg_times_save_ally_from_death,avg_takedowns_in_alcove,pct_of_games_first_blood_kill,pct_of_games_first_blood_assist,avg_takedowns_before_jungle_camps_spawn,avg_first_takedown_time,avg_summoner_spell1_casts_per_game,avg_summoner_spell2_casts_per_game,pct_of_matches_with_ghost,pct_of_matches_with_cleanse,pct_of_matches_with_smite,pct_of_matches_with_teleport,pct_of_matches_with_exhaust,pct_of_matches_with_ignite,pct_of_matches_with_heal,pct_of_matches_with_flash,pct_of_matches_with_barrier,avg_champ_exp_at_game_end,avg_champ_level_at_game_end,avg_gold_earned_per_game,avg_gold_per_minute,avg_gold_spent,avg_bounty_lvl,avg_bounty_gold,pct_of_games_with_early_lane_phase_gold_exp_adv,pct_of_games_with_lanephase_gold_exp_adv,avg_max_level_lead_over_lane_opp,avg_minions_killed,avg_minions_killed_by_10_mins,avg_max_cs_lead_over_lane_opponent,avg_consumables_purchased,avg_number_of_items_purchased,total_games_fastest_item_completion,avg_items_completed,avg_ally_jungle_minions_killed,avg_enemy_jungle_minions_killed,avg_enemy_jungle_cs_differential_early,avg_jungle_monsters_cs,avg_buffs_stolen,avg_initial_buff_count,avg_epic_monster_kills_within_30s_of_spawn,avg_initial_crab_count,avg_crabs_per_game,avg_jg_cs_before_10m,avg_jungler_kills_early_jungle,avg_jungler_early_kills_on_laners,avg_times_had_early_takedowns_in_all_lanes_as_laner,avg_jungler_takedowns_near_damaged_epic_monsters,avg_kills_with_help_from_epic_monster,avg_vision_score,avg_vision_score_per_min,avg_vision_score_advantage_over_lane_opponent,avg_stealth_wards_placed,avg_wards_placed,avg_wards_guarded,avg_control_wards_placed,avg_control_ward_time_coverage_in_river_or_enemy_half,avg_unseen_recalls,pct_of_games_with_highest_wards_killed,avg_wards_killed,avg_ward_takedowns,avg_ward_takedowns_before_20m,avg_times_2_wards_killed_with_1_sweeper,avg_control_wards_bought,pct_of_games_team_took_first_baron,avg_earliest_baron_by_team_time,pct_of_games_team_took_first_dragon,pct_of_games_team_took_first_inhib,pct_of_games_team_took_first_herald,pct_of_games_team_took_first_turret,avg_baron_kills_by_team,avg_hera

In [263]:
df = champion_x_role_df
# Creating features
df["total_role_games"] = df.groupby("team_position")["total_games_played_in_role"].transform("sum")
# Popularity metrics
df["popularity_in_role"] = df["total_games_played_in_role"]/df["total_role_games"]
df["role_popularity_for_champion"] = df["total_games_played_in_role"]/df["total_games_per_champion"]
# Damage done metrics
df["pct_magic_damage"] = df["avg_magic_damage_dealt_to_champions"]/df["avg_damage_dealt_to_champions"]
df["pct_physical_damage"] = df["avg_physical_damage_dealt_to_champions"]/df["avg_damage_dealt_to_champions"]
df["pct_true_damage"] = df["avg_true_damage_dealt_to_champions"]/df["avg_damage_dealt_to_champions"]
# Damage taken metrics
df["avg_damage_taken_per_death"] = df["avg_damage_taken"]/df["avg_deaths"]
# JUNGLE (REVISIT UPSTREAM)
df["avg_kills_on_other_lanes_early_as_laner"] = np.where( 
    df["team_position"] != "JUNGLE", 
    df["avg_kills_on_other_lanes_early_as_laner"], # As a laner, in a single game, get kills before 10 minutes outside your lane (anyone but your lane opponent)
    pd.NA
)
df["avg_times_had_early_takedowns_in_all_lanes_as_laner"] = np.where( 
    df["team_position"] != "JUNGLE", 
    df["avg_times_had_early_takedowns_in_all_lanes_as_laner"], # As a laner, in a single game, get kills before 10 minutes outside your lane (anyone but your lane opponent)
    pd.NA
)
df["avg_kills_early_jungle_as_jungler"] = np.where(
    df["team_position"] == "JUNGLE", 
    df["avg_jungler_kills_early_jungle"], # As jungler, get kills on the enemy jungler in their own jungle before 10 minutes
    pd.NA
)
df["avg_early_kills_on_laners_as_jungler"] = np.where(
    df["team_position"] == "JUNGLE", 
    df["avg_jungler_early_kills_on_laners"], # As jungler, get kills on top lane, mid lane, bot lane, or support players before 10 minutes
    pd.NA
)
df["pct_of_matches_with_jungle"] = np.where(
    df["team_position"] == "JUNGLE", 
    df["pct_of_matches_with_jungle"], # As jungler, get kills on top lane, mid lane, bot lane, or support players before 10 minutes
    pd.NA
)
df["avg_enemy_jungle_cs_differential_early"] = np.where(
    df["team_position"] == "JUNGLE", 
    df["avg_enemy_jungle_cs_differential_early"], # As jungler, get kills on top lane, mid lane, bot lane, or support players before 10 minutes
    pd.NA
)
df["avg_supp_quest_completion_time"] = np.where(
    df["team_position"] == "UTILITY", 
    df["avg_supp_quest_completion_time"], # As a support, average support quest completion time
    pd.NA
)

# Structures
df["avg_individual_tower_assists"] = df["avg_individual_tower_takedowns"] - df["avg_individual_tower_kills"]


In [264]:
labels = ["team_position", "champion_name"]

jungle_specific_features = ["pct_of_matches_with_jungle", "avg_kills_early_jungle_as_jungler",
                            "avg_early_kills_on_laners_as_jungler", "avg_enemy_jungle_cs_differential_early",
                            "avg_buffs_stolen", "avg_jungle_monsters_cs"]
laners_specific_features = ["avg_kills_on_other_lanes_early_as_laner", "avg_times_had_early_takedowns_in_all_lanes_as_laner",
                            "pct_of_games_indiv_killed_1st_tower", "pct_of_games_team_took_first_dragon", "pct_of_games_had_drag_takedown",
                            "pct_of_matches_with_mana", "avg_vision_score_advantage_over_lane_opponent",]

raw_clustering_features = [
    # Core 
    "avg_kills",
    "avg_deaths",
    "avg_assists",
    "avg_kill_participation",
    # Damage dealt
    "average_damage_per_minute",
    "avg_damage_dealt_to_champions",
    # Damage taken
    "avg_damage_self_mitigated",
    "avg_times_survived_three_immobilizes_in_fight",
    # CC
    "avg_time_ccing_others",
    "avg_times_applied_cc_on_others",
    "avg_enemy_champion_immobilizations",
    # Healing and shielding + Protecting
    "avg_total_healing",
    # Ability casts + Skillshots
    "avg_spell4_casts",
    "avg_ability_uses",
    "avg_skillshots_dodged",
    "avg_skillshots_landed_early_game",
    "avg_skillshots_hit",
    # Picks
    "avg_times_immobilize_and_kill_with_ally",
    "avg_times_pick_kill_with_ally",
    "avg_times_knock_enemy_into_team_and_kill",
    # Kill types
    "avg_kills_near_enemy_turret",
    "avg_outnumbered_kills",
    "avg_quick_solo_kills",
    "avg_solo_kills",
    # Laning kills
    "avg_takedowns_after_gaining_lvl_advantage",
    "avg_takedowns_in_alcove",
    "avg_first_takedown_time",
    # Summoner spells
    "pct_of_matches_with_ghost",
    "pct_of_matches_with_teleport",
    "pct_of_matches_with_ignite",
    # Gold and XP
    "avg_champ_exp_at_game_end",
    "avg_gold_earned_per_game",
    "avg_gold_per_minute",
    "pct_of_games_with_early_lane_phase_gold_exp_adv", # End the early laning phase (7 minutes) with 20% more gold and experience than your role opponent on Summoner's Rift
    "pct_of_games_with_lanephase_gold_exp_adv", # End the laning phase (14 minutes) with 20% more gold and experience than your role opponent 
    "avg_max_level_lead_over_lane_opp",
    # CS
    "avg_minions_killed",
    "avg_minions_killed_by_10_mins",
    "avg_cs_per_minute",
    "avg_max_cs_lead_over_lane_opponent",
    # Items
    "avg_number_of_items_purchased",
    "total_games_fastest_item_completion",
    # Jungle CS
    "avg_enemy_jungle_minions_killed",
    "avg_jungle_monsters_cs",
    "avg_buffs_stolen",
    "avg_initial_buff_count",
    "avg_epic_monster_kills_within_30s_of_spawn",
    "avg_initial_crab_count",
    "avg_crabs_per_game",
    "avg_enemy_jungle_cs_differential_early",
    # Vision Score
    "avg_vision_score_advantage_over_lane_opponent",
    "avg_ward_takedowns_before_20m",
    # Objs
    "pct_of_games_team_took_first_dragon",
    "avg_individual_rift_herald_takedowns",
    "avg_individual_dragon_takedowns",
    "avg_elder_dragon_kills_w_opposing_soul",
    "avg_indiv_dmg_dealt_to_buildings",
    "avg_indiv_turret_plates_taken",
    "pct_of_games_indiv_killed_1st_tower",
    "avg_individual_solo_towers_kills_late_game",
    "avg_individual_inhibitor_kills",
    "avg_individual_void_monster_kills",
    "pct_of_games_had_drag_takedown",
    # Game length
    "avg_time_played_per_game_minutes",
    # Multikills + Killing sprees
    "pct_of_games_first_blood_kill",
    "pct_of_games_first_blood_assist",
    "avg_number_of_multikills",
    "avg_quadrakills",
    "avg_multikills_with_one_spell",
    "avg_killing_sprees",
    # Mejais
    "percent_of_games_with_fully_stacked_mejais", ## new
    # Item tags
    "pct_of_matches_with_magic_resist",
    "pct_of_matches_with_jungle",
    "pct_of_matches_with_tenacity",
    "pct_of_matches_with_lane",
    "pct_of_matches_with_spell_block",
    "pct_of_matches_with_armor_penetration",
    "pct_of_matches_with_life_steal",
    "pct_of_matches_with_health",
    "pct_of_matches_with_attack_speed",
    "pct_of_matches_with_critical_strike",
    "pct_of_matches_with_ability_haste",
    "pct_of_matches_with_on_hit",
    "pct_of_matches_with_damage",
    "pct_of_matches_with_slow",
    "pct_of_matches_with_spell_damage",
    "pct_of_matches_with_spell_vamp",
    "pct_of_matches_with_aura",
    "pct_of_matches_with_magic_penetration",
    "pct_of_matches_with_cooldown_reduction",
    "pct_of_matches_with_armor",
    "pct_of_matches_with_health_regen",
    "pct_of_matches_with_mana",
    "pct_of_matches_with_nonboots_movement"
]


derived_clustering_features = [
    "popularity_in_role", 
    "role_popularity_for_champion",
    "pct_magic_damage",
    "pct_physical_damage",
    "pct_true_damage",
    "avg_damage_taken_per_death",
    "avg_effective_heal_and_shield",
    "avg_kills_on_other_lanes_early_as_laner",
    "avg_kills_early_jungle_as_jungler",
    "avg_early_kills_on_laners_as_jungler",
    "avg_times_had_early_takedowns_in_all_lanes_as_laner",
    "avg_control_ward_time_coverage_in_river_or_enemy_half",
    "avg_first_turret_kill_time_by_team",
    "avg_individual_tower_assists"
]

potential_features = [
    "pct_of_games_with_highest_damage_dealt",
    "avg_largest_critical_strike",
    "avg_pct_damage_taken_in_team",
    "avg_times_took_large_damage_survived", # Compare with 'avg_damage_taken_per_death' and likely choose 1
    "avg_supp_quest_completion_time",
    "pct_of_games_first_blood_kill",
    "pct_of_games_first_blood_assist",
    "avg_jungler_takedowns_near_damaged_epic_monsters",
    "avg_kills_with_help_from_epic_monster",
    "avg_control_wards_placed",
    "avg_ward_takedowns",
    "pct_of_games_team_took_first_baron",
    "pct_of_games_team_took_first_inhib",
    "avg_shortest_time_to_ace_from_1st_takedown",
    "pct_of_games_individual_killed_nexus",
    "avg_individual_dmg_dealt_to_objectives",
    "avg_individual_dragon_kills",
    "avg_objectives_stolen",
    "pct_of_games_had_drag_takedown_min_5_to_7",
    "pct_of_games_had_drag_takedown_min_7_to_11",
    "pct_of_games_had_drag_takedown_min_11_to_15",
    "pct_of_games_had_drag_takedown_min_15_plus",
    "avg_legendary_count",
    "avg_full_team_takedowns",
    "avg_pentakills",
    "avg_triplekills",
    "avg_doublekills",
    "avg_vision_score_per_min",
    "avg_pct_damage_dealt_in_team",
    "avg_heals_on_teammate",
    "avg_dmg_shielded_on_team",
    "avg_dragon_kills_by_team",
    "pct_of_games_team_took_first_herald",
    "pct_of_games_team_took_first_turret",
    "pct_of_games_individual_had_1st_turret_assist",
    "avg_individual_tower_kills",
    "pct_of_games_first_turret_taken_by_team",
    "avg_jg_cs_before_10m",
    "avg_times_save_ally_from_death",
]

low_value_features = [
    "total_wins",
    "avg_takedowns",
    "avg_total_damage_dealt",
    "avg_total_magic_damage_dealt",
    "avg_total_physical_damage_dealt",
    "avg_total_true_damage_dealt",
    "avg_damage_taken",
    "avg_magic_damage_taken",
    "avg_physical_damage_taken",
    "avg_true_damage_taken",
    "avg_times_survived_single_digit_hp",
    "pct_of_games_with_highest_crowd_control_score",
    "avg_total_units_healed",
    "total_games_completed_supp_quest_first",
    "avg_longest_time_spent_alive",
    "avg_time_spent_dead",
    "avg_spell1_casts",
    "avg_spell2_casts",
    "avg_spell3_casts",
    "avg_times_dodged_skillshot_in_small_window",
    "avg_times_got_kill_after_hidden_with_ally",
    "avg_kills_under_own_turret",
    "avg_multikills_after_aggressive_flash",
    "avg_times_outnumbered_nexus_kill",
    "avg_times_quick_cleanse",
    "avg_takedowns_before_jungle_camps_spawn",
    "avg_summoner_spell1_casts_per_game",
    "avg_summoner_spell2_casts_per_game",
    "pct_of_matches_with_smite",
    "pct_of_matches_with_cleanse",
    "pct_of_matches_with_heal",
    "pct_of_matches_with_flash",
    "pct_of_matches_with_barrier",
    "pct_of_matches_with_exhaust",
    "avg_champ_level_at_game_end",
    "avg_gold_spent",
    "avg_bounty_lvl",
    "avg_bounty_gold",
    "avg_consumables_purchased",
    "avg_items_completed",
    "avg_ally_jungle_minions_killed",
    "avg_vision_score",
    "avg_wards_placed",
    "avg_unseen_recalls",
    "pct_of_games_with_highest_wards_killed",
    "avg_wards_killed",
    "avg_times_2_wards_killed_with_1_sweeper",
    "avg_control_wards_bought",
    "avg_earliest_baron_by_team_time",
    "avg_baron_kills_by_team",
    "avg_herald_kills_by_team",
    "pct_of_games_with_perfect_drag_soul_taken",
    "avg_elder_dragon_kills_by_team",
    "avg_inhib_kills_by_team",
    "avg_tower_kills_by_team",
    "avg_inhibs_lost_by_team",
    "pct_of_games_with_nexus_lost_by_team",
    "avg_turrets_lost_by_team",
    "avg_total_team_champ_kills",
    "avg_team_aces_before_15_by_team",
    "avg_flawless_aces_by_team",
    "avg_max_kill_deficit",
    "pct_of_games_that_are_perfect_games",
    "avg_indiv_dmg_dealth_to_turrets",
    "pct_of_games_individual_took_1st_tower_quick",
    "avg_turrets_killed_before_plates_fell",
    "avg_indiv_towers_taken_w_rift_herald",
    "avg_individual_inhibitor_takedowns",
    "avg_individual_nexus_takedowns",
    "pct_of_games_with_open_nexus",
    "avg_individual_baron_kills",
    "avg_individual_baron_takedowns",
    "avg_individual_dragon_takedowns",
    "avg_individual_rift_herald_takedowns",
    "avg_objectives_stolen_assists",
    "avg_epic_monster_steals",
    "avg_epic_monster_steals_without_smite",
    "avg_epic_monsters_killed_near_enemy_jgler",
    "avg_earliest_drag_takedown",
    "avg_game_length",
    "pct_of_games_ended_in_early_ff",
    "pct_of_games_ended_in_ff",
    "pct_of_games_team_ffd",
    "avg_largest_multikill",
    "avg_killing_sprees2",
    "avg_largest_killing_spee",
    "avg_unreal_kills",
    "avg_elder_drag_multikills",
    "avg_times_blast_cone_enemy",
    "pct_of_games_danced_with_rift_herald",
    "avg_double_aces",
    "avg_fist_bump_participations",
    "avg_mejai_full_stack_time", # Needs fixing upstream
    "avg_outer_turret_executes_before_10m",
    "avg_takedowns_in_enemy_fountain",
    "mode_individual_position", # Double check if these positional cols are useful
    "mode_lane",
    "mode_role",
    "mode_team_position",
    "pct_of_games_played_champ_select_position",
    "role_play_rate",
    "kda",
    "win_rate",
    "avg_cs",
    "pct_games_first_to_complete_item",
    "avg_mejai_full_stack_time", ## new
    "avg_stealth_wards_placed",
    "pct_of_matches_with_trinket",
    "pct_of_matches_with_consumable",
    "pct_of_matches_with_vision",
    "pct_of_matches_with_boots",
    "avg_wards_guarded",
    "avg_12_assist_streaks",
    "pct_of_matches_with_gold_per",
    "avg_times_killed_champ_took_full_team_damage_and_survived",
    "avg_individual_solo_baron_kills",
    "pct_of_matches_with_active",
    "pct_of_matches_with_stealth",
    "pct_of_matches_with_mana_regen",
]

In [265]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define role mappings
ROLE_CONFIG = {
    'JUNGLE': {
        'exclude_features': laners_specific_features
    },
    'TOP': {
        'exclude_features': jungle_specific_features
    },
    'MID': {
        'exclude_features': jungle_specific_features
    },
    'BOTTOM': {
        'exclude_features': jungle_specific_features
    },
    'UTILITY': {
        'exclude_features': jungle_specific_features
    }
}

def filter_role_data(
    df: pd.DataFrame,
    role: str,
    labels: list[str],
    raw_clustering_features: list[str],
    derived_clustering_features: list[str],
    exclude_features: list[str],
    min_popularity: float = 0.1
) -> pd.DataFrame:
    """
    1. Select only label + clustering features, minus any excluded.
    2. Filter to the given role.
    3. Filter out low-popularity champions.
    4. Report & fill any missing values.
    """
    # 1) Build the “keep” list, ensuring we never drop the two needed filters
    required = {"team_position", "role_popularity_for_champion"}
    all_feats = set(labels) | set(raw_clustering_features) | set(derived_clustering_features) | required
    keep_cols = [
        col for col in all_feats
        if col in df.columns and col not in exclude_features
    ]
    
    # 2) Subset & filter by role
    role_df = df.loc[df["team_position"] == role, keep_cols].copy()
    
    # 3) Filter by popularity
    role_df = role_df.loc[
        role_df["role_popularity_for_champion"] > min_popularity
    ].copy()
    
    # 4) Check for missing values and fill them
    missing = role_df.isna().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    if not missing.empty:
        print(f"\n[filter_role_data] {role} — missing values:\n{missing}\n")
    role_df.fillna(0, inplace=True)
    
    return role_df

# Process all roles
filtered_dfs = {}
for role, config in ROLE_CONFIG.items():
    filtered_dfs[role] = filter_role_data(df, role, labels, raw_clustering_features, derived_clustering_features, config['exclude_features'])
    print(f'{role}: {filtered_dfs[role].shape}')

# Access individual dataframes
filtered_jungle_df = filtered_dfs['JUNGLE']
filtered_top_df = filtered_dfs['TOP']
filtered_mid_df = filtered_dfs['MID']
filtered_bot_df = filtered_dfs['BOTTOM']
filtered_supp_df = filtered_dfs['UTILITY']

# Display jungle data as example
filtered_jungle_df.head()

JUNGLE: (55, 103)
TOP: (61, 104)
MID: (0, 104)
BOTTOM: (33, 104)
UTILITY: (49, 104)


,team_position,avg_champ_exp_at_game_end,avg_crabs_per_game,avg_indiv_dmg_dealt_to_buildings,pct_of_games_first_blood_kill,avg_elder_dragon_kills_w_opposing_soul,avg_damage_self_mitigated,avg_quick_solo_kills,pct_of_matches_with_spell_damage,pct_of_games_first_blood_assist,pct_of_matches_with_ability_haste,avg_times_pick_kill_with_ally,pct_of_matches_with_aura,avg_ability_uses,avg_times_applied_cc_on_others,avg_individual_inhibitor_kills,avg_times_survived_three_immobilizes_in_fight,avg_outnumbered_kills,avg_early_kills_on_laners_as_jungler,pct_of_matches_with_ghost,pct_of_matches_with_health,avg_quadrakills,avg_kills_early_jungle_as_jungler,avg_jungle_monsters_cs,avg_skillshots_landed_early_game,avg_initial_crab_count,avg_assists,avg_epic_monster_kills_within_30s_of_spawn,avg_gold_earned_per_game,avg_ward_takedowns_before_20m,avg_time_ccing_others,avg_solo_kills,pct_of_matches_with_damage,avg_spell4_casts,avg_minions_killed_by_10_mins,pct_of_matches_with_slow,avg_initial_buff_count,avg_takedowns_after_gaining_lvl_advantage,avg_times_immobilize_and_kill_with_ally,pct_of_matches_with_attack_speed,avg_multikills_with_one_spell,avg_first_turret_kill_time_by_team,average_damage_per_minute,pct_of_matches_with_magic_resist,avg_cs_per_minute,avg_individual_tower_assists,avg_time_played_per_game_minutes,pct_of_matches_with_jungle,avg_skillshots_hit,pct_of_matches_with_tenacity,pct_of_matches_with_life_steal,avg_killing_sprees,champion_name,avg_total_healing,avg_enemy_jungle_minions_killed,pct_of_matches_with_critical_strike,avg_kills_near_enemy_turret,pct_of_matches_with_nonboots_movement,avg_minions_killed,avg_damage_dealt_to_champions,pct_of_games_with_lanephase_gold_exp_adv,pct_of_matches_with_health_regen,pct_of_matches_with_ignite,avg_max_level_lead_over_lane_opp,pct_of_matches_with_spell_block,avg_control_ward_time_coverage_in_river_or_enemy_half,pct_of_matches_with_lane,pct_of_matches_with_armor_penetration,avg_skillshots_dodged,avg_enemy_champion_immobilizations,avg_deaths,avg_max_cs_lead_over_lane_opponent,avg_number_of_items_purchased,pct_of_matches_with_cooldown_reduction,pct_magic_damage,avg_times_knock_enemy_into_team_and_kill,pct_true_damage,total_games_fastest_item_completion,avg_indiv_turret_plates_taken,avg_effective_heal_and_shield,role_popularity_for_champion,avg_first_takedown_time,avg_kill_participation,avg_individual_solo_towers_kills_late_game,avg_damage_taken_per_death,pct_of_games_with_early_lane_phase_gold_exp_adv,avg_individual_dragon_takedowns,pct_of_matches_with_teleport,pct_physical_damage,pct_of_matches_with_armor,popularity_in_role,percent_of_games_with_fully_stacked_mejais,avg_buffs_stolen,avg_kills,pct_of_matches_with_on_hit,avg_individual_void_monster_kills,avg_gold_per_minute,avg_number_of_multikills,avg_takedowns_in_alcove,pct_of_matches_with_spell_vamp,pct_of_matches_with_magic_penetration,avg_individual_rift_herald_takedowns,avg_enemy_jungle_cs_differential_early
24,JUNGLE,14073.423636,3.257262,1346.464349,13.138204,0.008803,36772.586708,0.000000,0.582768,8.186620,13.669224,11.999340,0.693420,149.319322,364.969410,0.138864,2.274868,1.310960,0.0,5.941901,54.809678,0.003081,0.0,137.223151,0.000000,0.976452,7.845070,0.397447,11648.685079,1.689701,30.458627,1.258803,59.316908,8.413292,2.759683,49.491000,2.000220,0.011884,8.640625,48.111537,0.098812,836.150311,580.765481,4.337563,5.780924,0.738996,28.477626,3.747418,0.015185,6.203895,26.032753,1.756602,Warwick,24468.186620,10.886004,0.066391,1.556778,23.281204,27.403829,17472.658671,7.025921,14.207731,1.386444,1.436795,13.868398,0.566096,4.308055,1.106521,29.324164,36.193882,6.463688,16.611919,20.233055,12.304515,0.464094,0.010123,0.052364,287,0.242077,0.201062,0.562515,6.293134,0.531881,1.051546,6578.570597,4.925441,1.864657,0.044014,0.483488,20.573916,0.016447,0.000880,0.919454,7.577245,35.224255,3.926056,405.859726,0.702465,0.189040,0.287695,0.531130,0.440801,0.0
50,JUNGLE,13577.585085,2.994995,998.979980,5.905906,0.006006,52665.775776,0.000000,1.0

In [266]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def cluster_and_report(df: pd.DataFrame, n_clusters: int = 8, random_state: int = 42):
    """
    Runs KMeans on all non-label columns, assigns clusters,
    then prints out cluster memberships and:
      - overall feature importance (range across centroids)
      - per-cluster drivers (deviation from global centroid)
    """
    # Make a clean copy and drop any existing cluster column
    df = df.copy()
    df.drop(columns=['cluster'], errors='ignore', inplace=True)

    # 1) Identify feature columns (exclude labels only)
    label_cols   = ['champion_name', 'team_position']
    feature_cols = [c for c in df.columns if c not in label_cols]

    # 2) Scale & cluster
    X_scaled = StandardScaler().fit_transform(df[feature_cols])
    km = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    df['cluster'] = km.fit_predict(X_scaled)

    # 3) Print cluster memberships
    print("=== Top Champion Clusters ===")
    for i in range(n_clusters):
        members = df.loc[df['cluster'] == i, 'champion_name']
        print(f"\nCluster {i} ({len(members)} champions):")
        print(members.to_string(index=False))

    # 4) Overall feature importance (range of centroid values)
    centroids     = km.cluster_centers_                  # shape (k, n_features)
    importance    = np.ptp(centroids, axis=0)            # peak-to-peak per feature
    importance_df = pd.DataFrame({
        'feature':    feature_cols,
        'importance': importance
    }).sort_values('importance', ascending=False)

    print("\n=== Top 10 Most Poggers Features ===")
    print(importance_df.head(10).to_string(index=False))

    # 5) Top drivers per cluster (deviation from global mean)
    global_center = centroids.mean(axis=0)

    print("\n=== Top 5 Drivers per Cluster ===")
    for i, center in enumerate(centroids):
        diffs   = center - global_center
        top_idx = np.argsort(np.abs(diffs))[::-1][:5]
        print(f"\nCluster {i} drivers:")
        for j in top_idx:
            feat = feature_cols[j]
            val  = diffs[j]
            print(f"  {feat}: {val:+.3f}")

In [269]:
cluster_and_report(filtered_jungle_df, n_clusters=8)

=== Top Champion Clusters ===

Cluster 0 (6 champions):
 Yorick
Belveth
   Olaf
   Gwen
Trundle
    Jax

Cluster 1 (7 champions):
Sejuani
 Maokai
  Amumu
    Zac
Skarner
 Rammus
   Nunu

Cluster 2 (14 champions):
   Warwick
MonkeyKing
    RekSai
  Pantheon
     Briar
   DrMundo
     Poppy
  Nocturne
  JarvanIV
    Darius
   XinZhao
  Volibear
        Vi
    LeeSin

Cluster 3 (4 champions):
Nidalee
 Gragas
Evelynn
  Elise

Cluster 4 (4 champions):
MasterYi
 Kindred
   Viego
  Graves

Cluster 5 (9 champions):
   Kayn
  Shaco
 Rengar
    Zed
 Qiyana
 Khazix
Hecarim
Naafiri
  Talon

Cluster 6 (1 champions):
Ivern

Cluster 7 (10 champions):
        Udyr
      Lillia
        Ekko
     Shyvana
     Taliyah
        Zyra
      Rumble
FiddleSticks
     Karthus
       Diana

=== Top 10 Most Poggers Features ===
                                              feature  importance
                        avg_effective_heal_and_shield    7.085079
                         avg_individual_tower_assists   